In [2]:
import os
# 安装最新版tts-autolabel
os.system("pip install -U tts-autolabel -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html")
# os.system("pip install -U tts-autolabel -i https://mirrors.aliyun.com/pypi/simple")
from modelscope.tools import run_auto_label

import requests
import zipfile
import IPython.display as ipd 

from modelscope.metainfo import Trainers
from modelscope.trainers import build_trainer
from modelscope.utils.audio.audio_utils import TtsTrainType
from modelscope.models.audio.tts import SambertHifigan
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 32.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 38.4 MB/s eta 0:00:00


DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fairseq 0.12.2 requires hydra-core<1.1,>=1.0.7, but you have hydra-core 1.3.2 which is incompatible.
fairseq 0.12.2 requires omegaconf<2.1, but you have omegaconf 2.3.0 which is incompatible.
onnx 1.14.1 requires protobuf>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
tensorflow 2.13.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.0 which is incompatible.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.


2023-11-09 21:06:38,497 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-11-09 21:06:38,501 - modelscope - INFO - TensorFlow version 2.13.0 Found.
2023-11-09 21:06:38,502 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2023-11-09 21:06:38,502 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2023-11-09 21:06:38,699 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 a8a545858e67690dd394730b698bbb8b and a total number of 945 components indexed
/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def download_file(url, save_path):
    """
    指定的Url下载文件,到指定的save_path;
    """
    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)
    print(f"文件下载完成,存盘到{save_path}！")

In [4]:
# 下载例子男声
url = "https://isv-data.oss-cn-hangzhou.aliyuncs.com/ics/MaaS/TTS/download_files/test_male.zip"
save_path = "/mnt/workspace/test_male.zip"
download_file(url, save_path)

文件下载完成,存盘到/mnt/workspace/test_male.zip！


In [5]:
# 定义压缩文件的路径
zip_path = '/mnt/workspace/test_male.zip'
# 指定解压缩的目标路径
target_path = '/mnt/workspace/test_wavs/'  # 解压缩后的文件将会被放在这个目录下

with zipfile.ZipFile(zip_path ,'r') as zf:
    for file in zf.namelist():
        file_path = file.split('/')
        if file_path[0] == 'test_male' and  file_path[1] != "":
            zf.extract(file, target_path)
# 移动到target_path,删除空目录:            
os.system(f'mv {target_path}test_male/*.* {target_path}')
os.system(f'rm -d {target_path}test_male')        

0

In [9]:
# 将录音文件*.m4a格式,转化为16k单通道wav格式
import soundfile as sf
import librosa 

In [51]:
sample_rate = 16000
for sn in range(1,21):
    audio_path = f'./test_wavs/个性化声音训练_I{sn}.m4a'
    save_path = f'./test_wavs/01_{sn:0>6}.wav'
    audio, sr = librosa.load(audio_path, sr=sample_rate, mono=True)
    sf.write(save_path, audio,samplerate=sr)
    

/opt/conda/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/librosa/util/decora

In [52]:
# 新建TTS-AutoLabel工作文件夹output_training_data
os.system("mkdir /mnt/workspace/output_training_data")

input_wav = "./test_wavs/"
output_data = "./output_training_data/"

ret, report = run_auto_label(input_wav=input_wav, work_dir=output_data, resource_revision="v1.0.7")

mkdir: 无法创建目录 “/mnt/workspace/output_training_data”: 文件已存在
2023-11-09 22:33:42,101 - modelscope - INFO - Use user-specified model revision: v1.0.7
INFO:root:---  New folder /mnt/workspace/output_training_data/paragraph/prosody...  ---
INFO:root:---  OK  ---
INFO:root:---  New folder /mnt/workspace/output_training_data/wav...  ---
INFO:root:---  OK  ---
INFO:root:---  New folder /mnt/workspace/output_training_data/wav_decrease_sp...  ---
INFO:root:---  OK  ---
INFO:root:--- Remove /mnt/workspace/output_training_data/log folder!  ---
INFO:root:---  New folder /mnt/workspace/output_training_data/log...  ---
INFO:root:---  OK  ---
INFO:root:2023-11-09 22:33:51
INFO:root:TTS-AutoLabel version: 1.1.8
INFO:root:TTS-AutoLabel resource path: /mnt/workspace/.cache/modelscope/damo/speech_ptts_autolabel_16k/model
INFO:root:Target sampling rate: 16000
INFO:root:Input wav dir: /mnt/workspace/test_wavs
INFO:root:Output data dir: /mnt/workspace/output_training_data
INFO:root:wav_preprocess start...
IN

text.cc: festival_Text_init


100%|██████████| 20/20 [00:04<00:00,  4.27it/s]
INFO:root:pre-break recording in paragraph by vad.
INFO:root:Generate phone interval by fa align.
INFO:root:prosody_dir=/mnt/workspace/output_training_data/paragraph/prosody
INFO:root:FA processing...
INFO:root:---  New folder /mnt/workspace/output_training_data/raw_ali...  ---
INFO:root:---  OK  ---
INFO:root:---  New folder /mnt/workspace/output_training_data/raw_interval...  ---
INFO:root:---  OK  ---
  0%|          | 0/20 [00:00<?, ?it/s]INFO:root:wav: /mnt/workspace/output_training_data/wav_cut_16k/01_000011_S0000.wav, text: 不过 当他 看到 拇指 姑娘的 时候 他 马上 就 变得 高兴 起来 
avr_tok:39, max_tok:79, avr_wbtok:1, max_wbtok:4, avr_wetok:2, max_wetok:8, avr_we_arc 4 avr_lmcall:4 p4, avr_histbeam:150.000000, max_histbeam:150.000000, avr_wbhistbeam:100.000000, max_wbhistbeam:100.000000
  5%|▌         | 1/20 [00:00<00:02,  8.19it/s]INFO:root:wav: /mnt/workspace/output_training_data/wav_cut_16k/01_000018_S0000.wav, text: 狮子 还是 够不着 他 叫来了 狐狸 
avr_tok:36, max

In [53]:
# 基于PTTS-basemodel微调

pretrained_model_id = 'damo/speech_personal_sambert-hifigan_nsf_tts_zh-cn_pretrain_16k'
# pretrained_model_id = 'damo/speech_personal_sambert-hifigan_nsf_tts_zh-cn_pretrain_24k'

dataset_id = "./output_training_data/"
pretrain_work_dir = "./pretrain_work_dir/"

os.system("mkdir /mnt/workspace/pretrain_work_dir")
        
# 训练信息，用于指定需要训练哪个或哪些模型，这里展示AM和Vocoder模型皆进行训练
# 目前支持训练：TtsTrainType.TRAIN_TYPE_SAMBERT, TtsTrainType.TRAIN_TYPE_VOC
# 训练SAMBERT会以模型最新step作为基础进行finetune
train_info = {
    TtsTrainType.TRAIN_TYPE_SAMBERT: {  # 配置训练AM（sambert）模型
        'train_steps': 202,               # 训练多少个step 
        'save_interval_steps': 200,       # 每训练多少个step保存一次checkpoint
        'log_interval': 10               # 每训练多少个step打印一次训练日志
    }
}

# 配置训练参数，指定数据集，临时工作目录和train_info
kwargs = dict(
    model=pretrained_model_id,                  # 指定要finetune的模型
    model_revision = "v1.0.6",
    work_dir=pretrain_work_dir,                 # 指定临时工作目录
    train_dataset=dataset_id,                   # 指定数据集id
    train_type=train_info                       # 指定要训练类型及参数
)

trainer = build_trainer(Trainers.speech_kantts_trainer,
                        default_args=kwargs)

trainer.train()

2023-11-09 22:34:40,144 - modelscope - INFO - Use user-specified model revision: v1.0.6
Downloading: 100%|██████████| 4.07k/4.07k [00:00<00:00, 350kB/s]
2023-11-09 22:34:41,703 - modelscope - INFO - Use user-specified model revision: v1.0.6
Downloading: 100%|██████████| 138k/138k [00:00<00:00, 1.15MB/s]
Downloading: 100%|██████████| 288k/288k [00:00<00:00, 2.53MB/s]
Downloading: 100%|██████████| 327k/327k [00:00<00:00, 2.51MB/s]
Downloading: 100%|██████████| 535k/535k [00:00<00:00, 3.79MB/s]
Downloading: 100%|██████████| 506/506 [00:00<00:00, 153kB/s]
Downloading: 100%|██████████| 1.69M/1.69M [00:00<00:00, 7.70MB/s]
Downloading: 100%|██████████| 452k/452k [00:00<00:00, 2.64MB/s]
Downloading: 100%|██████████| 998k/998k [00:00<00:00, 5.80MB/s]
Downloading: 100%|█████████▉| 866M/866M [00:08<00:00, 102MB/s] 
Downloading: 100%|██████████| 150M/150M [00:05<00:00, 29.1MB/s] 
Downloading: 100%|██████████| 3.82k/3.82k [00:00<00:00, 327kB/s]
Downloading: 100%|██████████| 2.49k/2.49k [00:00<00:00

Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed


festival_initialize() called more than once
100%|██████████| 40/40 [00:00<00:00, 8114.74it/s]
INFO:root:TextScriptConvertor.process:
Save script to: ./pretrain_work_dir/data/Script.xml
INFO:root:TextScriptConvertor.process:
Save metafile to: ./pretrain_work_dir/data/raw_metafile.txt
INFO:root:[AudioProcessor] Initialize AudioProcessor.
INFO:root:[AudioProcessor] config params:
INFO:root:[AudioProcessor] wav_normalize: True
INFO:root:[AudioProcessor] trim_silence: True
INFO:root:[AudioProcessor] trim_silence_threshold_db: 60
INFO:root:[AudioProcessor] preemphasize: False
INFO:root:[AudioProcessor] sampling_rate: 16000
INFO:root:[AudioProcessor] hop_length: 200
INFO:root:[AudioProcessor] win_length: 1000
INFO:root:[AudioProcessor] n_fft: 2048
INFO:root:[AudioProcessor] n_mels: 80
INFO:root:[AudioProcessor] fmin: 0.0
INFO:root:[AudioProcessor] fmax: 8000.0
INFO:root:[AudioProcessor] phone_level_feature: True
INFO:root:[AudioProcessor] se_feature: True
INFO:root:[AudioProcessor] norm_type:

In [54]:
model_dir = os.path.abspath("./pretrain_work_dir")

custom_infer_abs = {
    'voice_name':
    'F7',
    'am_ckpt':
    os.path.join(model_dir, 'tmp_am', 'ckpt'),
    'am_config':
    os.path.join(model_dir, 'tmp_am', 'config.yaml'),
    'voc_ckpt':
    os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan', 'ckpt'),
    'voc_config':
    os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan',
             'config.yaml'),
    'audio_config':
    os.path.join(model_dir, 'data', 'audio_config.yaml'),
    'se_file':
    os.path.join(model_dir, 'data', 'se', 'se.npy')
}
kwargs = {'custom_ckpt': custom_infer_abs}

model_id = SambertHifigan(os.path.join(model_dir, "orig_model"), **kwargs)



2023-11-09 22:38:19,992 - modelscope - INFO - am_config=/mnt/workspace/pretrain_work_dir/tmp_am/config.yaml voc_config=/mnt/workspace/pretrain_work_dir/orig_model/basemodel_16k/hifigan/config.yaml
2023-11-09 22:38:19,992 - modelscope - INFO - audio_config=/mnt/workspace/pretrain_work_dir/data/audio_config.yaml
2023-11-09 22:38:19,993 - modelscope - INFO - am_ckpts=OrderedDict([(2400000, '/mnt/workspace/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400000.pth'), (2400200, '/mnt/workspace/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400200.pth')])
2023-11-09 22:38:19,993 - modelscope - INFO - voc_ckpts=OrderedDict([(2400000, '/mnt/workspace/pretrain_work_dir/orig_model/basemodel_16k/hifigan/ckpt/checkpoint_2400000.pth')])
2023-11-09 22:38:19,994 - modelscope - INFO - se_path=/mnt/workspace/pretrain_work_dir/data/se/se.npy se_model_path=/mnt/workspace/pretrain_work_dir/orig_model/se.onnx
2023-11-09 22:38:19,995 - modelscope - INFO - mvn_path=/mnt/workspace/pretrain_work_dir/orig_model/mvn.npy


Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed


festival_initialize() called more than once


In [60]:
inference = pipeline(task=Tasks.text_to_speech, model=model_id)
# input = '春眠不觉晓，处处闻啼鸟, 夜来风雨声，花落知多少。'
input = '君不见，黄河之水天上来，奔流到海不复回。君不见，高堂明镜悲白发，朝如青丝暮成雪。人生得意须尽欢，莫使金樽空对月。天生我材必有用，千金散尽还复来。烹羊宰牛且为乐，会须一饮三百杯。岑夫子，丹丘生，将进酒，杯莫停。与君歌一曲，请君为我倾耳听。钟鼓馔玉不足贵，但愿长醉不复醒。古来圣贤皆寂寞，惟有饮者留其名。陈王昔时宴平乐，斗酒十千恣欢谑。主人何为言少钱，径须沽取对君酌。五花马，千金裘，呼儿将出换美酒，与尔同销万古愁。'
output = inference(input=input)

ipd.Audio(output["output_wav"], rate=16000)

2023-11-09 22:50:12,481 - modelscope - WARNING - No preprocessor field found in cfg.
2023-11-09 22:50:12,482 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2023-11-09 22:50:12,483 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/mnt/workspace/pretrain_work_dir/orig_model'}. trying to build by task and model information.
2023-11-09 22:50:12,483 - modelscope - WARNING - No preprocessor key ('sambert-hifigan', 'text-to-speech') found in PREPROCESSOR_MAP, skip building preprocessor.


In [61]:
# 将pretrain_work_dir 压缩成一个压缩包:
pretrain_dir = "./pretrain_work_dir"
pretrain_zip_path = "./pretrain_work_dir.zip"
with zipfile.ZipFile(pretrain_zip_path, 'w') as z:
    for root, dirs, files in os.walk(pretrain_dir):
        for file in files:
            z.write(os.path.join(root,file))  
         